## Batch feature extraction of images

### Importing libraries

In [2]:
import numpy as np
import os
import cv2
import mahotas as mt

np.set_printoptions(suppress=True)

### Image files preparation

In [3]:
healthy_folder = "dataset/healthy2"
blight_folder = "dataset/infected2"
healthy_images = os.listdir(healthy_folder)
blight_images = os.listdir(blight_folder)

### Loading images and extracting features

In [4]:
def extract_features(rgb_img):
    features = []
    # prepare masked versions
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2HSV)
    ity_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
    # infected_area = np.where(mask == 1)

    # mean red
    red_img = rgb_img[:,:,2].flatten() / 255
    features.append(np.mean(red_img))
    
    # mean green
    grn_img = rgb_img[:,:,1].flatten() / 255
    features.append(np.mean(grn_img))

    # mean blue
    blu_img = rgb_img[:,:,0].flatten() / 255
    features.append(np.mean(blu_img))

    # mean hue
    hue_img = hsv_img[:,:,0].flatten()
    features.append(np.mean(hue_img))

    # std red
    features.append(np.std(red_img))

    # std green
    features.append(np.std(grn_img))

    # std blue
    features.append(np.std(blu_img))
    
    # texture features
    textures = mt.features.haralick(ity_img, return_mean=True)
    features.append(textures[0])
    features.append(textures[4])
    features.append(textures[7])
    features.append(textures[8])
    features.append(textures[9])

    return np.array(features)



In [5]:
X = []
y = []

for img_path in healthy_images:
    img = cv2.imread(os.path.join(healthy_folder, img_path))
    img = cv2.filter2D(img,-1,(np.ones((5,5),np.float32)/25))
    features = extract_features(img)
    X.append(features)
    y.append(0)  # 0 for healthy


for img_path in blight_images:
    img = cv2.imread(os.path.join(blight_folder, img_path))
    img = cv2.filter2D(img,-1,(np.ones((5,5),np.float32)/25))
    features = extract_features(img)
    X.append(features)
    y.append(1)  # 1 for blight

X = np.array(X)
y = np.array(y)

### Export dataset

In [6]:
np.savetxt('leaf_blight_dataset.csv', np.hstack((X,y[:,np.newaxis])), delimiter=",", fmt='%g')